### Using AWS SageMaker for Inference
On AWS sagemaker, using a pretrained Object Detection Model works best with tensorflow 2.8 and needs an updated version of sagemaker to use this version of tensorflow.

In [2]:
!pip install tensorflow_hub
%pip install -U sagemaker
!pip install tensorflow==2.8

  Using cached tensorflow_hub-0.12.0-py2.py3-none-any.whl (108 kB)
You should consider upgrading via the '/usr/local/bin/python3.8 -m pip install --upgrade pip' command.
  Using cached sagemaker-2.92.1.tar.gz (536 kB)
  Preparing metadata (setup.py) ... done
  Using cached attrs-20.3.0-py2.py3-none-any.whl (49 kB)
  Using cached importlib_metadata-1.7.0-py2.py3-none-any.whl (31 kB)
  Created wheel for sagemaker: filename=sagemaker-2.92.1-py2.py3-none-any.whl size=739509 sha256=cdd8338baa67ca2bd07dc39e3190acacfbeba309e1ced44eaa946d6ad1791be6
  Stored in directory: /root/.cache/pip/wheels/1c/33/f6/8e5902410891d990c867bcf79b42282e6cebab5477d8d8f309
Successfully built sagemaker
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.8.2
    Uninstalling importlib-metadata-4.8.2:
      Successfully uninstalled importlib-metadata-4.8.2
  Attempting uninstall: attrs
    Found existing installation: attrs 21.2.0
    Uninstalling attrs-21.2.0:
      Succ

Import the necessary libraries

In [3]:
import os
import pathlib

import matplotlib
import matplotlib.pyplot as plt

import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from six.moves.urllib.request import urlopen

import tensorflow as tf
import tensorflow_hub as hub
import sagemaker
import json
import cv2
tf.get_logger().setLevel('ERROR')

In [5]:
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
dataset_bucket = "swifty-datasets"

bucket = sess.default_bucket()

print("Bucket: {}".format(bucket))
print("SageMaker ver: " + sagemaker.__version__)
print("Tensorflow ver: " + tf.__version__)

Bucket: sagemaker-us-east-1-038469568353
SageMaker ver: 2.92.1
Tensorflow ver: 2.8.0


In [6]:
from sagemaker.tensorflow import TensorFlowModel

model = TensorFlowModel(model_data='s3://swifty-ai-models/other_models/object_detector.tar.gz', role=role,framework_version = '2.8')

predictor = model.deploy(initial_instance_count=1, instance_type='ml.m5.xlarge', endpoint_name='object-detection-endpoint-1')

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


ClientError: An error occurred (ValidationException) when calling the CreateEndpointConfig operation: Cannot create already existing endpoint configuration "arn:aws:sagemaker:us-east-1:038469568353:endpoint-config/object-detection-endpoint-1".

In [29]:
#Send the copy of the image arrat 
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: the file path to the image

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  image = None
  if(path.startswith('https')):
    response = urlopen(path)
    image_data = response.read()
    image_data = BytesIO(image_data)
    image = Image.open(image_data)
  else:
    image_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(image_data)) 
    #resize and sent this image as array to endpoint for inference
  basewidth = 360 #Resize to basewidth of 800 and resize it proportionally to the height
  wpercent = (basewidth/float(image.size[0]))
  hsize = int((float(image.size[1])*float(wpercent)))
  image = image.resize((basewidth,hsize), Image.ANTIALIAS)
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (1, im_height, im_width, 3)).astype(np.uint8)

In [30]:
image_path = '/root/AI-Projects/Fire_Detection/FireScene.jpg' 
!pwd

/root/AI-Projects/Fire_Detection


In [31]:
input_tensor = load_image_into_numpy_array(image_path)

In [32]:
#Send this to object detection model
input_data = {
  'instances': input_tensor.tolist()
}

In [33]:
data = json.dumps({"instances":input_tensor.tolist()})

In [34]:
print(input)

<bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x7f4a80c23370>>


In [35]:
import requests

In [36]:
#invoke API
headers = {"content-type":"application/json"}
json_response = requests.post('https://p5dv58hpub.execute-api.us-east-1.amazonaws.com/InitialStage/objectdataprediction',data=data,headers = headers)
result = json.loads(json_response.text)

In [ ]:
print(result)

In [38]:
print(result['predictions'][0]['detection_classes'])
print(result['predictions'][0]['detection_scores'])

[79.0, 79.0, 44.0, 51.0, 47.0, 51.0, 47.0, 79.0, 78.0, 47.0, 78.0, 78.0, 51.0, 81.0, 50.0, 81.0, 44.0, 86.0, 47.0, 86.0, 44.0, 81.0, 44.0, 82.0, 49.0, 44.0, 47.0, 78.0, 51.0, 82.0, 49.0, 81.0, 79.0, 85.0, 64.0, 82.0, 79.0, 50.0, 70.0, 86.0, 75.0, 80.0, 51.0, 44.0, 86.0, 85.0, 47.0, 79.0, 86.0, 81.0, 78.0, 44.0, 49.0, 50.0, 82.0, 78.0, 51.0, 80.0, 81.0, 50.0, 79.0, 44.0, 44.0, 75.0, 70.0, 51.0, 49.0, 82.0, 50.0, 44.0, 81.0, 51.0, 47.0, 79.0, 51.0, 84.0, 80.0, 82.0, 80.0, 50.0, 78.0, 51.0, 86.0, 47.0, 84.0, 51.0, 49.0, 1.0, 49.0, 90.0, 78.0, 50.0, 62.0, 86.0, 70.0, 64.0, 85.0, 81.0, 47.0, 82.0]
[0.736179411, 0.514530957, 0.373843729, 0.354321957, 0.307223022, 0.30539915, 0.303582639, 0.297191262, 0.293246269, 0.289334387, 0.287522376, 0.284987092, 0.272672772, 0.269670308, 0.262967229, 0.261257708, 0.258633733, 0.258322507, 0.255708188, 0.255621731, 0.251642942, 0.2452856, 0.24437505, 0.241588891, 0.241532624, 0.241177261, 0.23638925, 0.236201406, 0.222489595, 0.208358705, 0.206984639, 0

In [39]:
detection_scores= result['predictions'][0]['detection_scores']
detection_classes= result['predictions'][0]['detection_classes']

In [40]:
print(result['predictions'][0]['detection_classes'])

[79.0, 79.0, 44.0, 51.0, 47.0, 51.0, 47.0, 79.0, 78.0, 47.0, 78.0, 78.0, 51.0, 81.0, 50.0, 81.0, 44.0, 86.0, 47.0, 86.0, 44.0, 81.0, 44.0, 82.0, 49.0, 44.0, 47.0, 78.0, 51.0, 82.0, 49.0, 81.0, 79.0, 85.0, 64.0, 82.0, 79.0, 50.0, 70.0, 86.0, 75.0, 80.0, 51.0, 44.0, 86.0, 85.0, 47.0, 79.0, 86.0, 81.0, 78.0, 44.0, 49.0, 50.0, 82.0, 78.0, 51.0, 80.0, 81.0, 50.0, 79.0, 44.0, 44.0, 75.0, 70.0, 51.0, 49.0, 82.0, 50.0, 44.0, 81.0, 51.0, 47.0, 79.0, 51.0, 84.0, 80.0, 82.0, 80.0, 50.0, 78.0, 51.0, 86.0, 47.0, 84.0, 51.0, 49.0, 1.0, 49.0, 90.0, 78.0, 50.0, 62.0, 86.0, 70.0, 64.0, 85.0, 81.0, 47.0, 82.0]


In [41]:
def unique(list1):
     
    # insert the list to the set
    list_set = set(list1)
    # convert the set to the list
    unique_list = (list(list_set))
    print(unique_list)

In [42]:
unique(result['predictions'][0]['detection_classes'])

[1.0, 44.0, 47.0, 49.0, 50.0, 51.0, 62.0, 64.0, 70.0, 75.0, 78.0, 79.0, 80.0, 81.0, 82.0, 84.0, 85.0, 86.0, 90.0]


In [43]:
print(len((result['predictions'][0]['detection_scores'])))
print(len((result['predictions'][0]['detection_classes'])))

100
100


In [44]:
txt_file = open("cococlasses.txt", "r")
content = txt_file.read().split("\n")
print(content)

['background', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'street sign', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'hat', 'backpack', 'umbrella', 'shoe', 'eye glasses', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'plate', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'mirror', 'dining table', 'window', 'desk', 'toilet', 'door', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'blender', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush', 'hair brush']


In [45]:
print(len(content))

92


In [46]:
print(content[16])

bird


In [4]:
!ls /root/

AI-Projects


In [47]:
def convertToInt(list1):
    detected_classes=[]
    for i in range(len(list1)):
        detected_classes.append(int(list1[i]))
    return detected_classes 

In [48]:
detected_classes = convertToInt(result['predictions'][0]['detection_classes'])

In [49]:
def findTop10Classes(list1, list2, content):
    print("The system has detected the following objects:")
    print("Object \t\t\t Confidence")
    for i in range(10):
        print(content[list1[i]].capitalize(),"\t\t\t",list2[i]*100)

In [50]:
findTop10Classes(detected_classes,detection_scores,content)

The system has detected the following objects:
Object 			 Confidence
Oven 			 73.6179411
Oven 			 51.4530957
Bottle 			 37.3843729
Bowl 			 35.4321957
Cup 			 30.7223022
Bowl 			 30.539915
Cup 			 30.358263899999997
Oven 			 29.7191262
Microwave 			 29.324626900000002
Cup 			 28.933438700000004


### Delete End Point

In [12]:
sess.delete_endpoint(predictor.endpoint)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


### Without using AWS services

In [ ]:
detector = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2")

In [ ]:
image_path = 'https://upload.wikimedia.org/wikipedia/commons/0/09/The_smaller_British_birds_%288053836633%29.jpg'
image_np = load_image_into_numpy_array(image_path)

plt.figure(figsize=(24,32))
plt.imshow(image_np[0])
plt.show()


In [ ]:
# running inference
results = detector(image_np)
 #needs tf 2.8 to work

result = {key:value.numpy() for key,value in results.items()}
print(result.keys())

In [ ]:
print(image_np)

In [ ]:
print(result['detection_scores'])

In [ ]:
print(result['detection_classes'])
#Get image,
#Predict and return output from end point
#After predicting, Find way to add detection boxes to output and then save to AWS s3 on the image.